## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [1]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
import numpy as np
import pandas as pd
data_text = pd.read_csv("abcnews-date-text.csv")
data_text.drop('publish_date',axis = "columns",inplace = True)
index = []
for i in range(1103665):
    index.append(i)
data_text["index"] = index
#data_text = data_text[["index","headline_text"]]

Let's glance at the dataset:

In [2]:
data_text.head()
documents = data_text

In [3]:
'''
print the total number of documents
'''
documents.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1103665 entries, 0 to 1103664
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   headline_text  1103665 non-null  object
 1   index          1103665 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 16.8+ MB


In [4]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [5]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Meet
[nltk_data]     Mehta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [7]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense

go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [8]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [9]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
lm = WordNetLemmatizer()
ss = SnowballStemmer(language = "english")
def lemmatize_stemming(text):
    '''
        1) Lemmatize the text
        2) Then, stem the text
        3) return final text
    '''
    l1 = []
    l1.append(lm.lemmatize(text))
    l2 = []
    for i in l1:
        l2.append(stemmer.stem(i))
    f_text = ''.join(i for i in l2)
    return f_text 

# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
    return result

In [10]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]
print(doc_sample)
print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

rain helps dampen bushfires
Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [11]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = []
processed_docs.append(documents["headline_text"].apply(lambda y: preprocess(y)))

In [12]:
'''
Preview 'processed_docs' 
'''
processed_docs[:10]

[0                      [decid, communiti, broadcast, licenc]
 1                                         [wit, awar, defam]
 2                     [call, infrastructur, protect, summit]
 3                                [staff, aust, strike, rise]
 4                       [strike, affect, australian, travel]
                                  ...                        
 1103660    [ash, smith, warner, near, miss, liven, box, t...
 1103661                  [timelaps, brisban, year, firework]
 1103662                              [meant, kid, australia]
 1103663                     [papodopoulo, meet, mean, ausus]
 1103664           [georg, papadopoulo, trump, campaign, aid]
 Name: headline_text, Length: 1103665, dtype: object]

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [13]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
# Todo
dictionary = gensim.corpora.Dictionary(processed_docs[0])
#dictionary.cfs

In [14]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [15]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below = 15,no_above=0.1,keep_n = 100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [16]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
#  Todo
from gensim.corpora import Dictionary
#d1 = dictionary.cfs
#document = [(a,b) for a,b in d1.items()]
bow_corpus = [dictionary.doc2bow(document) for document in processed_docs[0]]

In [17]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(76, 1), (112, 1), (486, 1), (4041, 1)]

In [18]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 486 ("rain") appears 1 time.
Word 4041 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [19]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
from gensim.models import TfidfModel
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfModel(bow_corpus)
# TODO


In [20]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]
# TODO


In [21]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.589272278034924), (1, 0.3892842954875021), (2, 0.49648286011932197), (3, 0.5046986122767408)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [22]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
lda_model = gensim.models.LdaMulticore(bow_corpus,id2word = dictionary,num_topics = 10,workers = 6,alpha = 0.1,eta = 0.1)
# TODO

In [23]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.016*"australia" + 0.015*"world" + 0.014*"win" + 0.012*"final" + 0.009*"test" + 0.009*"weather" + 0.008*"leagu" + 0.008*"star" + 0.008*"podcast" + 0.007*"tour"


Topic: 1 
Words: 0.018*"govern" + 0.017*"flood" + 0.016*"fund" + 0.015*"jail" + 0.014*"water" + 0.010*"sentenc" + 0.010*"defend" + 0.010*"game" + 0.009*"public" + 0.009*"budget"


Topic: 2 
Words: 0.028*"year" + 0.022*"plan" + 0.015*"trump" + 0.013*"council" + 0.012*"fear" + 0.010*"meet" + 0.010*"news" + 0.009*"port" + 0.009*"job" + 0.008*"region"


Topic: 3 
Words: 0.016*"rise" + 0.014*"live" + 0.014*"high" + 0.010*"season" + 0.010*"take" + 0.010*"open" + 0.009*"rate" + 0.009*"action" + 0.008*"cattl" + 0.008*"shark"


Topic: 4 
Words: 0.023*"queensland" + 0.019*"north" + 0.019*"south" + 0.015*"west" + 0.013*"power" + 0.012*"coast" + 0.012*"resid" + 0.011*"forc" + 0.010*"leader" + 0.009*"chang"


Topic: 5 
Words: 0.021*"market" + 0.012*"share" + 0.011*"urg" + 0.010*"say" + 0.009*"gold" + 0.009*"busi" + 0.009*

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [24]:
'''
Define lda model using corpus_tfidf and store in lda_model_tfidf
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,id2word = dictionary,num_topics = 10,workers = 6,alpha = 0.1,eta = 0.1)
# TODO


In [25]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} \nWord: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.012*"drum" + 0.011*"hill" + 0.006*"capit" + 0.006*"broken" + 0.005*"februari" + 0.005*"homeless" + 0.005*"suicid" + 0.005*"donat" + 0.004*"biki" + 0.004*"caught"


Topic: 1 Word: 0.011*"crash" + 0.009*"polic" + 0.009*"assault" + 0.008*"charg" + 0.008*"grandstand" + 0.008*"woman" + 0.007*"royal" + 0.007*"fatal" + 0.006*"murder" + 0.006*"street"


Topic: 2 Word: 0.007*"climat" + 0.007*"friday" + 0.006*"septemb" + 0.006*"david" + 0.006*"violenc" + 0.006*"domest" + 0.006*"peter" + 0.005*"chang" + 0.005*"octob" + 0.005*"april"


Topic: 3 Word: 0.017*"interview" + 0.015*"trump" + 0.008*"australia" + 0.007*"donald" + 0.006*"test" + 0.006*"final" + 0.006*"michael" + 0.005*"leagu" + 0.005*"world" + 0.005*"ash"


Topic: 4 Word: 0.010*"weather" + 0.009*"polic" + 0.009*"charg" + 0.008*"sport" + 0.007*"miss" + 0.007*"drug" + 0.007*"court" + 0.007*"search" + 0.006*"monday" + 0.005*"june"


Topic: 5 Word: 0.009*"stab" + 0.009*"guilti" + 0.008*"turnbul" + 0.008*"plead" + 0.007*"live" 

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [26]:
'''
Text of sample document 4310
'''
processed_docs[0][4310]

['rain', 'help', 'dampen', 'bushfir']

In [27]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.4348948299884796	 
Topic: 0.016*"rise" + 0.014*"live" + 0.014*"high" + 0.010*"season" + 0.010*"take" + 0.010*"open" + 0.009*"rate" + 0.009*"action" + 0.008*"cattl" + 0.008*"shark"

Score: 0.4050457179546356	 
Topic: 0.016*"australia" + 0.015*"world" + 0.014*"win" + 0.012*"final" + 0.009*"test" + 0.009*"weather" + 0.008*"leagu" + 0.008*"star" + 0.008*"podcast" + 0.007*"tour"

Score: 0.02001050114631653	 
Topic: 0.021*"market" + 0.012*"share" + 0.011*"urg" + 0.010*"say" + 0.009*"gold" + 0.009*"busi" + 0.009*"campaign" + 0.008*"abbott" + 0.008*"hous" + 0.008*"council"

Score: 0.02000935934484005	 
Topic: 0.018*"govern" + 0.017*"flood" + 0.016*"fund" + 0.015*"jail" + 0.014*"water" + 0.010*"sentenc" + 0.010*"defend" + 0.010*"game" + 0.009*"public" + 0.009*"budget"

Score: 0.02000766061246395	 
Topic: 0.020*"rural" + 0.017*"report" + 0.015*"elect" + 0.011*"nation" + 0.010*"hill" + 0.010*"research" + 0.010*"vote" + 0.009*"health" + 0.009*"say" + 0.009*"hospit"

Score: 0.020007463172

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [28]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
for index, score in sorted(lda_model_tfidf[corpus_tfidf[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))
# Todo


Score: 0.6942076086997986	 
Topic: 0.009*"govern" + 0.006*"plan" + 0.005*"fund" + 0.005*"health" + 0.005*"council" + 0.005*"green" + 0.005*"labor" + 0.005*"say" + 0.005*"elect" + 0.005*"legal"

Score: 0.03399293124675751	 
Topic: 0.008*"marriag" + 0.006*"budget" + 0.006*"andrew" + 0.005*"price" + 0.005*"liber" + 0.005*"drought" + 0.005*"elect" + 0.005*"wrap" + 0.004*"burn" + 0.004*"speed"

Score: 0.033983781933784485	 
Topic: 0.010*"weather" + 0.009*"polic" + 0.009*"charg" + 0.008*"sport" + 0.007*"miss" + 0.007*"drug" + 0.007*"court" + 0.007*"search" + 0.006*"monday" + 0.005*"june"

Score: 0.03397827595472336	 
Topic: 0.007*"climat" + 0.007*"friday" + 0.006*"septemb" + 0.006*"david" + 0.006*"violenc" + 0.006*"domest" + 0.006*"peter" + 0.005*"chang" + 0.005*"octob" + 0.005*"april"

Score: 0.03397700935602188	 
Topic: 0.020*"rural" + 0.019*"countri" + 0.018*"hour" + 0.013*"market" + 0.012*"news" + 0.010*"podcast" + 0.008*"share" + 0.008*"busi" + 0.007*"christma" + 0.006*"asylum"

Score:

## Step 6: Testing model on unseen document ##

In [29]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6149164438247681	 Topic: 0.018*"govern" + 0.017*"flood" + 0.016*"fund" + 0.015*"jail" + 0.014*"water"
Score: 0.22496071457862854	 Topic: 0.020*"rural" + 0.017*"report" + 0.015*"elect" + 0.011*"nation" + 0.010*"hill"
Score: 0.020036231726408005	 Topic: 0.021*"market" + 0.012*"share" + 0.011*"urg" + 0.010*"say" + 0.009*"gold"
Score: 0.020021041855216026	 Topic: 0.034*"woman" + 0.031*"polic" + 0.020*"attack" + 0.018*"miss" + 0.015*"perth"
Score: 0.0200111735612154	 Topic: 0.016*"rise" + 0.014*"live" + 0.014*"high" + 0.010*"season" + 0.010*"take"
Score: 0.02001096121966839	 Topic: 0.023*"queensland" + 0.019*"north" + 0.019*"south" + 0.015*"west" + 0.013*"power"
Score: 0.020010899752378464	 Topic: 0.022*"charg" + 0.021*"murder" + 0.020*"countri" + 0.017*"hour" + 0.014*"australia"
Score: 0.020010879263281822	 Topic: 0.028*"year" + 0.022*"plan" + 0.015*"trump" + 0.013*"council" + 0.012*"fear"
Score: 0.02001085877418518	 Topic: 0.036*"interview" + 0.026*"crash" + 0.024*"child" + 0.019

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Append points here
- Need to make sure that all the data provided is related to the topic and prevent providing unwanted data
- Make use of a stopword library that removes maximum of the stopwords to ovoid them from hindering the accuracy of the model